All changes in this part should be made in `scheme_reader.py`

In Parts 1 and 2, we will develop the interpreter in several stages:
1. Reading Scheme expressions
2. Symbol evaluation
3. Calling built-in procedures
4. Definitions
5. Lambda expressions and procedure definition
6. Calling user-defined procedures
7. Evaluation of special forms

The first part of this project deals with reading and parsing user inout. Our reader will parse Scheme code into Python values with the following representations:

|Input Example | Scheme Expression Type | Our Internal Representation |
| --- | --- | --- |
|`scm> 1` | Numbers | Python's built-in `int` and `float` values |
|`scm> x` | Symbols | Python's built-in `string` values |
|`scm> #t` | Booleans (`#t`, `#f`) | Python's built-in `True`, `False` values|
|`scm> (+ 2 3)` | Combinations | Instances of the `Pair` class, defined in `scheme_reader.py` |
|`scm> nil` | `nil` | The `nil` object, defined in `scheme_reader.py` |

When we refer to combinations in this project, we are referring to both call expressions and special forms.

Make sure to read the [Implementation Overview](https://inst.eecs.berkeley.edu/~cs61a/fa18/proj/scheme/#implementation-overview) section to understand how the reader is broken up into parts.

## Implementation Overview

Here is a brief overview of each of the `Read-Eval-Print Loop` components in our interpreter. Refer to this section as we work through the project as a reminder of how all the small pieces fit together!

**1.** `Read`: This step parses user input (a string of Scheme code) into our interpreter's internal Python representation of Scheme expressions (e.g. Pairs).

`Lexical Analysis` has been already implemented in the `tokenize_lines` function in `scheme_tokens.py`. This function returns a `Buffer` (from `buffer.py`) of tokens. We don't need to read or understand the code for this step.

`Syntactic Analysis` happens in `scheme_reader.py`, in the `scheme_read` and `read_tail` functions. Together, these mutually recursive functions parse Scheme tokens into our interpreter's internal Python representation of Scheme expressions. We will complete both functions.

**2.** `Eval`: This step evalues Scheme expessions (represented in Python)

In our implementation, we store tokens ready to be parsed in `Buffer` instances. For example, a buffer containing the input,

In [ ]:
(+ 2 (2 . 3))

...would have the tokens,

In [ ]:
'(', '+', '(', 2, '.', 3, ')', ')'

See the doctests in `buffer.py` for more examples. We don't need to understand the code in this file.

We will write the parsing functionality, which consists of 2 mutually recursive functions `scheme_read` and `read_tail`. These functions each take in a single parameter, `src`, which is an instance of `Buffer`.

There are 2 methods defined in `buffer.py` that we'll use to interact with `src`:

**1.**`src.remove_front()`: mutates `src` by removing the **first** token in `src` and returns it. For the sake of simplicity, if we imagine `src` as a Python list such as,

In [ ]:
[4, '.', 3, ')']

`src.remove_front()` will return `4`, and `src` will be left with,

In [ ]:
['.', 3, ')']

**2.**`src.current()`: returns the **first** token in `src` without removing it. For example, if `src` currently contains the tokens,

In [ ]:
[4, '.', 3, ')']

then `src.current()` will return `4` but `src` will remain the same.

## Problem 1

#### WWSD - Case 2

In [ ]:
>>> from scheme_reader import *
>>> tokens = tokenize_lines(["(+ 1 ", "(23 4)) ("])
>>> src = Buffer(tokens)
>>> src.current()
'('
>>> src.remove_front()
'('
>>> src.current()
'+'
>>> src.remove_front()
'+'
>>> src.remove_front()
1
>>> scheme_read(src)  # Returns and removes the next complete expression in src
Pair(23, Pair(4, nil))
>>> src.current()
')'

#### WWSD - Case 3

In [ ]:
>>> from scheme_reader import *
>>> scheme_read(Buffer(tokenize_lines(['(18 6)'])))
Pair(18, Pair(6, nil))
>>> read_line('(18 6)') # Shorter version of above
Pair(18, Pair(6, nil))

#### WWSD - Case 4

In [ ]:
>>> from scheme_reader import *
>>> read_tail(Buffer(tokenize_lines([')'])))
nil
>>> read_tail(Buffer(tokenize_lines(['1 2 3)'])))
Pair(1, Pair(2, Pair(3, nil)))
>>> read_tail(Buffer(tokenize_lines(['2 (3 4))'])))
Pair(2, Pair(Pair(3, Pair(4, nil)), nil))

#### WWSD - Case 7

In [ ]:
>>> from scheme_reader import *
>>> read_line("(+ (- 2 3) 1)")
# Choose the number of the correct choice:
# 0) Pair('+', Pair(Pair('-', Pair(2, Pair(3, nil))), Pair(1, nil)))
# 1) Pair('+', Pair('-', Pair(2, Pair(3, Pair(1, nil)))))
# 2) Pair('+', Pair('-', Pair(2, Pair(3, nil))), Pair(1, nil))
2

#### Strategy - `scheme_read`

If we look at the `scheme_read` function, it takes the first token and assigns it to `val.

In [ ]:
val = src.remove_front()

According to the problem description,

"If the current token is the string `'nil'`, return the `nil` object."

If we observe a few lines above the `scheme_read(src)` function, we'll see the following,

In [ ]:
nil = nil()

This means the `nil` variable is bound to the `nil` object! This part is then straightforward.

In [ ]:
if val == 'nil':
    return nil

Then if the current token is `(`, we `return` the result of calling `read_tail` on the rest of `src`. Note that `src` is already changed since when we assign `val` to `src.remove_front()`, Python removes the first token.

In [ ]:
elif val == '(':
    return read_tail(src)

#### Strategy - `read_tail`

"If the token is `)`, then we've reached the end of the list or pair. Remove this token from the buffer and return the `nil` object."

We can remove the current token from the buffer using the `src.remove_front()` method, then we just return the `nil` object.

In [ ]:
elif src.current() == ')':
    src.remove_front()
    return nil

"If one of the above cases apply, the next token is the operator in a combination, e.g. `src` contains `+ 2 3)`.

The problem description might seems complicated, but the implementation is actually straightforward. We just return a `Pair` object where the first element is the result of calling `scheme_read(src)`, and the second element is the result of calling `read_tail(src)`. Note that the `Pair` class definition is present in the first few lines within `scheme_reader.py`.

In [ ]:
else:
    return Pair(scheme_read(src), read_tail(src))

## Problem 2

#### WWSD - Case 1

In [ ]:
>>> from scheme_reader import *
>>> read_line("(a . b)")
# Choose the number of the correct choice:
# 0) Pair('a', 'b', nil)
# 1) Pair('a', Pair('b', nil))
# 2) SyntaxError
# 3) Pair('a', 'b')
# 4) Pair('a', Pair('b'))
3

In [ ]:
>>> from scheme_reader import *
>>> read_line("(a b . c)")
# Choose the number of the correct choice:
# 0) Pair('a', Pair('b', Pair('c')))
# 1) Pair('a', Pair('b', 'c'))
# 2) Pair('a', 'b', 'c')
# 3) SyntaxError
# 4) Pair('a', Pair('b', Pair('c', nil)))
1

In [ ]:
>>> from scheme_reader import *
>>> read_line("(a b . c d)")
# Choose the number of the correct choice:
# 0) Pair('a', Pair('b', Pair('c', 'd')))
# 1) Pair('a', Pair('b', Pair('c', Pair('d', nil))))
# 2) SyntaxError
# 3) Pair('a', Pair('b', 'c'))
2

In [ ]:
>>> from scheme_reader import *
>>> read_line("(a . (b . (c . ())))")
# Choose the number of the correct choice:
# 0) SyntaxError
# 1) Pair('a', Pair('b', Pair('c', nil)))
# 2) Pair('a', 'b', 'c')
# 3) Pair('a', Pair('b', Pair('c', Pair(nil, nil))))
1

In [ ]:
>>> from scheme_reader import *
>>> read_line("(a . ((b . (c))))")
# Choose the number of the correct choice:
# 0) Pair('a', Pair('b', Pair('c')), nil)
# 1) Pair('a', Pair(Pair('b', Pair('c', nil)), nil), nil)
# 2) Pair('a', Pair('b', Pair('c', nil)), nil)
# 3) Pair('a', Pair(Pair('b', Pair('c', nil)), nil))
3

#### Strategy

As described by the `hint`, we add support for dotted pairs by simply verifying that only one element follows a dot. This can be done by reading the expression after the `.`,

In [ ]:
src.remove_front()
to_be_returned = scheme_read(src)

Then check if the next token (after reading the expression) is a `)`. If not, raise a `SyntaxError`. otherwise, return the result of reading the expression.

In [ ]:
if not src.remove_front() == ')':
    raise SyntaxError
else:
    return to_be_returned